In [1]:
function createGraphFromFile(filename)
	f = open(filename)
	lines = readlines(f)
	firstLine = split(lines[1])
	n = parse(Int64, firstLine[1]) #número de vértices
	g = parse(Int64, firstLine[2]) #número de grupos
	secondLine = split(lines[2])
    L = zeros(g) #limite inferior do grupo j
    U = zeros(g) #limite superior do grupo j
    for j=1:g
    	L[j] = parse(Float64, secondLine[j*2-1])
    	U[j] = parse(Float64, secondLine[j*2])
    end
    p = zeros(n) #peso do vértice i
    thirdLine = split(lines[3])
    for i=1:n
    	p[i] = parse(Float64, thirdLine[i])
    end
    numberOfLines = size(lines)[1]
    A = zeros(n,n) #matriz de adjacência em que cada elemento representa o peso da aresta entre o vértice u e v
    for lineIndex=4:numberOfLines
    	currentLine = split(lines[lineIndex])
    	u = parse(Int64, currentLine[1]) + 1
        v = parse(Int64, currentLine[2]) + 1
        A[u,v] = parse(Float64, currentLine[3])
    end
    return n, g, p, L, U, A
end

n, g, p, L, U, A = createGraphFromFile("D:/UFRGS/Quinto Semestre/Otimização Combinatória/Trabalho final/gbmv240_02.ins")

(240, 12, [9.0, 1.0, 8.0, 8.0, 8.0, 8.0, 6.0, 9.0, 3.0, 1.0  …  8.0, 4.0, 5.0, 2.0, 4.0, 6.0, 3.0, 10.0, 2.0, 4.0], [75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0], [125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0, 125.0], [0.0 17.154 … 63.107 69.454; 0.0 0.0 … 8.412 12.936; … ; 0.0 0.0 … 0.0 58.988; 0.0 0.0 … 0.0 0.0])

In [2]:
using JuMP
using GLPKMathProgInterface
m = Model(solver=GLPKSolverMIP(msg_lev=GLPK.MSG_ALL, tm_lim=3600000, out_frq=1))

@variable(m, x[1:n, 1:g], Bin) #g é o número de grupos
@variable(m, c[1:n, 1:n, 1:g], Bin) #g é o número de grupos
@objective( m, Max, sum( c[i,k,j]*A[i,k] for i in 1:n for k in 1:n for j in 1:g ) )
@constraints(m, begin
[i in 1:n], sum(x[i,j] for j in 1:g)==1
[j in 1:g], L[j]<=sum( x[i,j]*p[i] for i in 1:n )<=U[j]
[i in 1:n, k in 1:n, j in 1:g], c[i,k,j]<=(x[i,j]+x[k,j])/2 # isso daqui garante que vou ter tanto c[1,2,j] quanto c[2,1,j], então não preciso me preocupar com qual vértice escolher pra i e qual pra k na função objetivo

end)
solve(m)

GLPK Integer Optimizer, v4.52
691452 rows, 694080 columns, 2076480 non-zeros
694080 integer variables, all of which are binary
Integer optimization begins...
+347938: mip =     not found yet <=              +inf        (1; 0)
|348500: obj =  1.436540466e+006  infeas = 2.184e-008 (0)
|348769: obj =  1.436540466e+006  infeas = 1.151e-008 (0)
+348769: mip =     not found yet <=  1.436540466e+006        (2; 0)
Time used: 3224.8 secs.  Memory used: 569.6 Mb.
|349500: obj =  1.436540466e+006  infeas = 1.429e-008 (5252)
|350000: obj =  1.436540466e+006  infeas = 7.531e-009 (5176)
|350500: obj =  1.436540466e+006  infeas = 6.293e-009 (4785)
|351000: obj =  1.436540466e+006  infeas = 5.017e-008 (4484)
|351500: obj =  1.436540466e+006  infeas = 6.864e-009 (4078)
|352000: obj =  1.436528373e+006  infeas = 6.700e-008 (3828)
|352500: obj =  1.436496987e+006  infeas = 5.979e-009 (3503)
|353000: obj =  1.436333762e+006  infeas = 8.843e-011 (3265)
|353500: obj =  1.436051736e+006  infeas = 2.387e-010 

:UserLimit

In [3]:
getobjectivevalue(m)

NaN